## Using twitter documentation


In [1]:
import os
from dotenv import load_dotenv
import requests
import requests_oauthlib
import json

In [2]:
load_dotenv()
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
ACCESS_SECRET = os.getenv("ACCESS_SECRET")
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

In [3]:
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN,
ACCESS_SECRET)

## Choose user

In [15]:
user = "realDonaldTrump"

## 1. Get twitter mentions

In [4]:
def get_tweet_mentions(user, date):
    url = f'https://api.twitter.com/1.1/search/tweets.json?q=%40{user}&count=100&tweet_mode=extended&until={date}'
    response = requests.get(url, auth=my_auth, stream=True)
    print(url, response)
    return response.json()

#should add count, result type: recent/popular

In [5]:
mentions = get_tweet_mentions(user, "2020-07-24")

https://api.twitter.com/1.1/search/tweets.json?q=%40realDonaldTrump&count=100&tweet_mode=extended&until=2020-07-24 <Response [200]>


In [6]:
tweet_mentions = []
for i in range(len(mentions["statuses"])):
    try: 
        tweet_mentions.append(mentions["statuses"][i]['retweeted_status']['full_text'])
    except: 
        tweet_mentions.append(mentions["statuses"][i]["full_text"])

In [7]:
tweet_mentions

['@realDonaldTrump This is what you do to suburban housewives. \n\n https://t.co/SN0hWaV6v7',
 '@realDonaldTrump Barbara Walters &amp; Donald Trump \nFull interview https://t.co/q2DBA3nIrN',
 'Wow — what a giant FUCK YOU to @realDonaldTrump!! 🖕\nhttps://t.co/vBMiKPYpQc',
 'WELCOME BACK BASEBALL! https://t.co/dhLSfNkI7L',
 '@brithume @realDonaldTrump Did someone hack Humes account?   That’s the first thing I have seen him write that makes sense.',
 '@WillieGarson @DannyZuker @realDonaldTrump WELCOME BACK BASEBALL',
 '@realDonaldTrump Wear your masks!',
 'Wow — what a giant FUCK YOU to @realDonaldTrump!! 🖕\nhttps://t.co/vBMiKPYpQc',
 "@realDonaldTrump Obama was the biggest criminal to set foot in the White House, Clinton, 2nd. \n\nI thank God every day that you stepped you, Mr. President. @realDonaldTrump \nYou sure didn't have to give up your life, but you sure did to save ours.",
 'So it’s too dangerous in Florida to hold a @gop Convention, but not too dangerous in Florida to reopen sc

In [8]:
# Retweets get cut with this code
"""tweet_mentions = []
for i in range(len(j["statuses"])):
    #count verified opinions x10
    
    if j["statuses"][i]["user"]["verified"]==False: 
        tweet_mentions.append(j["statuses"][i]["full_text"])
    elif j["statuses"][i]["user"]["verified"]==True: 
        tweet_mentions.extend([j["statuses"][i]["full_text"] for l in range(10)])"""

'tweet_mentions = []\nfor i in range(len(j["statuses"])):\n    #count verified opinions x10\n    \n    if j["statuses"][i]["user"]["verified"]==False: \n        tweet_mentions.append(j["statuses"][i]["full_text"])\n    elif j["statuses"][i]["user"]["verified"]==True: \n        tweet_mentions.extend([j["statuses"][i]["full_text"] for l in range(10)])'

Parameters:
lang ==> en
count ==> 100
until ==> (Returns tweets created before the given date. Date should be formatted as YYYY-MM-DD. Keep in mind that the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week)

## 2. Get tweets from user

In [9]:
def get_tweets(user):
    url = f'https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name={user}&tweet_mode=extended&count=100'
    response = requests.get(url, auth=my_auth, stream=True)
    print(url, response)
    return response.json()

In [10]:
user_t = get_tweets(user)

https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name=realDonaldTrump&tweet_mode=extended&count=100 <Response [200]>


In [11]:
tweets = {}
for i in range(len(user_t)):
    try:
        tweets[user_t[i]["id_str"]]= user_t[i]['retweeted_status']['full_text']
    except: 
        tweets[user_t[i]["id_str"]]= user_t[i]["full_text"]

In [34]:
list(tweets.items())[70]

('1286993270816378881',
 'Trump just said "if schools do not reopen the funding should go to parents." https://t.co/DQPpoMF8bK')

## 3. Get replies for a tweet

In [20]:
def get_replies(user, tweet_id):
    url = f'https://api.twitter.com/1.1/search/tweets.json?q=%40{user}&count=100&tweet_mode=extended&in_reply_to_status_id_str={tweet_id}'
    response = requests.get(url, auth=my_auth, stream=True)
    responses =  response.json()
    tweet_responses = []
    for i in range(len(responses["statuses"])):
        try: 
            tweet_responses.append(responses["statuses"][i]['retweeted_status']['full_text'])
        except: 
            tweet_responses.append(responses["statuses"][i]["full_text"])
    return tweet_responses

In [35]:
url = f'https://api.twitter.com/1.1/search/tweets.json?q=%40{user}&count=100&tweet_mode=extended&in_reply_to_status_id_str=1286993270816378881'
response = requests.get(url, auth=my_auth, stream=True)
print(response.json())


{'errors': [{'message': 'Rate limit exceeded', 'code': 88}]}


## 4. Create data list

In [23]:
data = []
for key, value in tweets.items():
    d = {}
    d["id"] = key
    d["tweet"] = value
    d["responses"] = get_replies(user, key)
    data.append(d)

KeyError: 'statuses'

In [26]:
len(data)

71

In [27]:
data

[{'id': '1287561862272430080',
  'tweet': 'How the heck did this happen?\nhttps://t.co/D4ALvgIHwY',
  'responses': ['@realDonaldTrump If ballots need to be recounted T least there will be a paper trail.',
   'The Trump Campaign has more ENTHUSIASM, according to many, than any campaign in the history of our great Country - Even more than 2016. Biden has NONE! The Silent Majority will speak on NOVEMBER THIRD!!! Fake Suppression Polls &amp; Fake News will not save the Radical Left.',
   '@rosepet19239091 @RKJ65 @realDonaldTrump Doubtful they predicted district gerrymandering that ensured the victories of moronic partisan scum either.',
   "Former U.S Navy SEAL Dr. Dan Barkhuff knows @realDonaldTrump isn't a conservative — he's weak.\n\nAnd he's the most easily fixable problem in America today. https://t.co/Ep5cCdY0pw",
   '@realDonaldTrump No The Trump Administration is',
   'Trump’s wall goes down in high winds. The amount of money @realDonaldTrump has wasted in the past three and a half

## Extracting Topics

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [15]:
def word_exctraction(lst):
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(lst) 
    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())
    all_words = df.sum().sort_values(ascending=[False]).to_dict()   
    stop_words = set(stopwords.words('english'))
    tokens_clean = {key: value for (key, value) in all_words.items() if key not in stop_words}
    return (tokens_clean)

In [16]:
word_exctraction(tweets)

{'realdonaldtrump': 78,
 'rt': 48,
 'https': 18,
 'co': 18,
 'people': 17,
 'trump': 11,
 'china': 8,
 'mcnaughtonart': 8,
 'lauraingram': 7,
 'tuckercarlson': 7,
 'seanhannity': 7,
 'banned': 7,
 'nancy': 7,
 'said': 7,
 'crazy': 7,
 'infected': 7,
 'dbongino': 7,
 'erictrump': 7,
 'pelosi': 7,
 'mistake': 6,
 'made': 6,
 'january': 6,
 'entering': 6,
 'lie': 5,
 'today': 5,
 'amp': 5,
 'country': 5,
 'gop': 5,
 'joe': 4,
 'de': 4,
 'look': 4,
 'golfing': 4,
 'let': 4,
 'president': 4,
 'campaign': 4,
 'never': 4,
 'america': 4,
 'enthusiasm': 3,
 'day': 3,
 'tell': 3,
 'many': 3,
 'one': 3,
 'call': 3,
 'office': 3,
 'city': 3,
 'great': 3,
 'really': 3,
 'senatemajldr': 3,
 'game': 3,
 'scottpresler': 3,
 'party': 3,
 'like': 3,
 'wrong': 3,
 'without': 3,
 'believe': 3,
 'biafra': 2,
 'white': 2,
 'republican': 2,
 'resign': 2,
 'right': 2,
 'montaga': 2,
 'week': 2,
 'new': 2,
 'way': 2,
 'kansas': 2,
 'vote': 2,
 'could': 2,
 'incredibly': 2,
 'however': 2,
 'vp': 2,
 'gopchairwo

## Extracting Vader Sentiment Analysis

In [83]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(sentence): 
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']

def sentiment_averages_vader(lst):
    l = []
    for e in lst: 
        l.append(sentiment_scores(e))
    return sum(l)/len(l)

In [89]:
sentiment_averages_vader(tweets)

0.04805555555555555

In [85]:
sentiment_averages_vader(tweets)

0.035160714285714295

ModuleNotFoundError: No module named 'sentiment'